**_DTM_** : Document Term Matrix   
**_TF-IDF_** : Term Frequency-Inverse Document Frequency
- 텍스트 표현 방법의 종류
- 정보 검색과 텍스트 마이닝 분야에서 주로 사용

---------

# 1. 다양한 단어의 표현 방법
## 1.1 단어의 표현 방법
1) 국소 표현(Local Representation) 방법 / 이산(Discrete) 표현
- 해당 단어 그 자체만 보고 특정값을 매핑하여 단어를 표현
- 단어의 의미, 뉘앙스 표현 X
- ex) puppy, cute, lovely 각각 숫자 맵핑

2) 분산 표현(Distributed Representation) 방법 / 연속(Continuous) 표현
- 해당 단어를 표현하고자 주변을 참고하여 단어를 표현
- 단어의 뉘앙스 표현 O
- ex) puppy라는 단어는 cute, lovely한 느낌이다로 단어를 정의

## 1.2 단어 표현의 카테고리화
4장 - BoW, DTM, IF-IDF    
9장 - 워드 임베딩, Word2Vec, FastText, GloVe

![그림1](./04-1.png)

# 2. Bag of Words(BoW)
빈도수 기반의 단어 표현 방법. 텍스트 데이터의 수치화 표현 방법.   
단어의 등장 순서를 고려하지 않는다.

BoW를 만드는 과정
(1) 각 단어에 고유한 정수 인덱스를 부여  # 단어 집합 생성.
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

## 2.1 한국어 예제
문서1) 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.   
문서2) 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.   
문서3) 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

In [1]:
# 사용자 함수 생성
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:  
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)  
            # BoW에 전부 기본값 1을 넣는다.
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
            bow[index] = bow[index] + 1

    return word_to_index, bow

In [2]:
# 문서1
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


> 인덱스 1값인 '가'와 4값인 '물가상승률'은 2번씩 언급되었으므로 BoW의 값이 2이다.

In [3]:
# 문서2
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'

vocab, bow = build_bag_of_words(doc2)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [4]:
# 문서3 = 문서1 + 문서2
doc3 = doc1 + ' ' + doc2
vocab, bow = build_bag_of_words(doc3)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


* BoW는 여러 문서의 단어 집합을 합친 뒤에, 해당 단어 집합에 대한 각 문서의 BoW를 구할 수도 있다.

> BoW는 주로 **분류 문제나 여러 문서 간의 유사도를 구하는 문제**에 주로 쓰인다.   
> 가령,  '미분', '방정식', '부등식'과 같은 단어가 자주 등장한다면 수학 관련 문서로 분류할 수 있다.

## 2.2 CounterVectorizer 클래스로 BoW 만들기
- 단어의 빈도를 Count하여 Vector로 만들어준다.
- 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식한다. (ex. I는 토큰 X)
- 주의) 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다. -> 영어의 경우 문제 없지만, 한국어에 적용시 BoW가 제대로 만들어지지 않는다.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


## 2.3 불용어를 제거한 BoW 만들기
BoW 사용 이유는 문서 내에서 단어의 등장 빈도수를 보겠다는 것인데, 이것은 결국 텍스트 내에서 어떤 단어들이 중요한지를 보겠다는 의미이다.   
따라서 BoW를 만들때 불용어를 제거하면 자연어 처리의 정확도를 높일 수 있을 것이다.

영어의 BoW를 만들기 위해 사용하는 CountVectorizer는 불용어를 지정하면, 불용어는 제외하고 BoW를 만들 수 있도록 불용어 제거 기능을 지원하고 있다.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [8]:
# 1) 사용자가 직접 정의한 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [9]:
# 2) CountVectorizer에서 제공하는 자체 불용어 사용
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


In [10]:
# 3) NLTK에서 지원하는 불용어 사용
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 3. 문서 단어 행렬(Document-Term Matrix, DTM)
- 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것   
  (각 문서에 대한 BoW를 하나의 행렬로 만든 것)
- 행과 열을 반대로 선택하면 TDM이라고 부르기도 한다.
- 장점) 문서들을 서로 비교할 수 있도록 수치화할 수 있다.
- 필요에 따라서는 형태소 분석기로 단어 토큰화를 수행하고, 불용어에 해당되는 조사들 또한 제거하여 더 정제된 DTM을 만들 수도 있을 것

< DTM의 한계 >
1) 희소 표현(Sparse representation)   
- 공간적 낭비와 계산 리소스를 증가시킨다. (원-핫 벡터와 동일한 단점..)
    
2) 단순 빈도 수 기반 접근   
- ex) 불용어인 the는 어떤 문서이든 자주 등장. but 유사도 비교 시 문서1~3에서 동일하게 the의 빈도수가 높다고 해서 유사한 문서하고 판단해서는 안 된다.   
- DTM에 불용어와 중요단어에 대해 가중치는 주는 아이디어를 적용한 게 TF-IDF이다!!

# 4. TF-IDF(단어 빈도-역문서 빈도, Term Frequency-Inverse Document Frequency)
- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 우선 DTM을 만든 후, TF-IDF 가중치를 부여
- 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 사용


- TF-IDF는 TF와 IDF를 곱한 값을 의미한다.

## 4.1 기본개념
문서를 d, 단어를 t, 문서의 총 개수를 n이라 할 때
1) **tf(d,t)** : 특정 문서 d에서의 특정 단어 t의 등장 횟수 ( = DTM의 값 )   
- 중요도와 비례

2) df(t) : 특정 단어 t가 등장한 문서의 수. (각 문서에서 몇 번 등장했는지는 중요하지 X)   
- 중요도와 반비례   

3) **idf(d, t)** : df(t)에 반비례하는 수.
![그림2](./04-2.png) 
- df에 역수를 취하는 이유 : 중요도와 비례하는 tf값과 곱하기 위해   
  (df가 클수록 중요도가 낮다. 즉 idf가 클수록 중요도가 높다.)
- log(보통 자연로그 ln 사용)를 쓰는 이유 : 역수를 취한 값이 기하급수적으로 커지는 것을 방지하기 위함
- 분모에 1 더하는 이유 : 분모가 0이 되는 상황을 방지하기 위함
- **idf값이 클수록 특정 문서에서만 단어 t가 등장한다.**
- 여러 문서에서 등장한 단어의 가중치를 낮추는 역할을 해준다.

TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, **_특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다._**  또한 TF-IDF값은 **중요도와 비례**한다.

## 4.2 파이썬으로 TF-IDF 직접 구현하기

In [11]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [13]:
# tf, idf, tf-idf 구하는 함수 구현

N = len(docs)  # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [18]:
# DTM을 DF에 저장하여 출력 = TF
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [19]:
# 각 단어에 대한 IDF값 구하기
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [21]:
# TF-IDF 행렬 출력
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_


,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


> 위의 기본적인 식을 바탕으로 한 구현에는 몇 가지 문제점이 존재하기 때문에 실제 TF-IDF 구현을 제공하고 있는 많은 머신 러닝 패키지들은 약간씩 조정된 식을 사용한다.

## 4.3 사이킷런을 이용한 DTM과 TF-IDF 실습

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())   ## DTM 완성! = TF

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)   

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [25]:
# TF-IDF를 자동 계산해주는 TfidfVectorizer 이용해보기
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)  # 말뭉치 학습
print(tfidfv.transform(corpus).toarray())   # TF-IDF 완성!!
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
